<a href="https://colab.research.google.com/github/MJMortensonWarwick/ADA/blob/main/3_3_fine_tuning_a_hugging_face_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning a Huggingface Transformer
In this Notebook we will download a transformer model (based on BERT), fine tune it on a new dataset (the Yelp reviews dataset). We will need to classify the data between one and five stars.

We start with some imports. Firstly, we will install accelerate ...<br>
**IMPORTANT!! After running this line of code please click "Runtime" > "Restart session" (or "Runtime" > "Restart session and run all" if you must)**

In [1]:
!pip install accelerate -U

And we can install the rest as normal (no restart required):

In [2]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset['train'][0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'label': 4,
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}

Now we'll tokenise the data using BERT embeddings

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

We only need a smaller set of this data for the demo so will take 1,000 random examples for training and the same for validation (no test this time as its just an example and its painfully slow):

In [5]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
val_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

We can now conver this into the Keras format:

In [6]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [7]:
tf_train_dataset = train_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = val_dataset.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:401: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


We will use a general purpose Huggingface model to classify our data:

In [8]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Finally we can (fine-)train the model in the usual fashion. We will only run for 5 epochs (as its slow) and you can ignore the AutoGraph warning:

In [9]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=5)

Epoch 1/5


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
125/125 [==============================] - 280s 2s/step - loss: 1.8438 - sparse_categorical_accuracy: 0.1791 - val_loss: 1.6631 - val_sparse_categorical_accuracy: 0.1821
Epoch 2/5
125/125 [==============================] - 159s 1s/step - loss: 1.7855 - sparse_categorical_accuracy: 0.1839 - val_loss: 1.7167 - val_sparse_categorical_accuracy: 0.1877
Epoch 3/5
125/125 [==============================] - 159s 1s/step - loss: 1.8009 - sparse_categorical_accuracy: 0.1899 - val_loss: 1.9334 - val_sparse_categorical_accuracy: 0.1870
Epoch 4/5
125/125 [==============================] - 199s 2s/step - loss: 1.7880 - sparse_categorical_accuracy: 0.1863 - val_loss: 1.6996 - val_sparse_categorical_accuracy: 0.1892
Epoch 5/5
125/125 [==============================] - 200s 2s/step - loss: 1.7726 - sparse_categorical_accuracy: 0.1922 - val_loss: 1.8653 - val_sparse_c

It is fairly clear that we need more training but we can see the accuracy creeping up! And that's the module! 👊